In [2]:
from bs4 import BeautifulSoup
import requests
import time
import os

In [3]:
DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [4]:
years = [2020, 2021, 2022, 2023, 2024]
full_urls_main = [] 

for year in years:
    url_main = f'https://www.basketball-reference.com/leagues/NBA_{year}_games.html'
    response_main = requests.get(url_main)
    soup_main = BeautifulSoup(response_main.content, 'html.parser')

    months_listed = soup_main.find_all('div', class_='filter')

    for months in months_listed:
        sub_months = months.find_all('a')
        for sub_month in sub_months:
            month_href = sub_month.get('href')
            if month_href:
                full_url_main = f'https://www.basketball-reference.com{month_href}'
                full_urls_main.append(full_url_main)  

In [ ]:
suburls = []

for url in full_urls_main:
    time.sleep(5) 
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table')

        if table:
            rows = table.find_all('tr')[1:]

            for row in rows:
                columns = row.find_all('td', class_='center')
                for column in columns:
                    subcols = column.find_all('a')
                    for subcol in subcols:
                        suburl = subcol.get('href')
                        if suburl:
                            full_url = f'https://www.basketball-reference.com{suburl}'
                            suburls.append(full_url)

            print(f"Found {len(suburls)} game URLs on {url}")
        else:
            print(f"No table found on {url}")

    except Exception as e:
        print(f"Error fetching data from {url}: {e}")

In [ ]:
for url in suburls:
    time.sleep(2) 
    try:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        content_div = soup.find(id='content')
        if content_div:
            with open(save_path, 'w+', encoding='utf-8') as f:
                f.write(content_div.prettify())
      
            print(f"Saved main content of {url} to {save_path}")
        else:
            print(f"No #content found on {url}")

    except Exception as e:
        print(f"Error saving data from {url}: {e}")